In [1]:
import pandas as pd
import numpy as np
import re
from keras.models import Model
from keras.layers.core import Dropout, Dense, Flatten, Activation
from keras.layers.embeddings import Embedding
from keras.layers.wrappers import Bidirectional
from keras.layers.recurrent import GRU
from keras.layers.pooling import MaxPooling2D
from keras.layers import Merge,Input


Using TensorFlow backend.


In [2]:
df = pd.read_csv('quora_duplicate_questions.tsv',sep='\t',encoding='utf8')

In [3]:
df = df.dropna()

In [4]:
import spacy
#from gensim.utils import lemmatize
nlp = spacy.load('en')

In [5]:
def clean_data(string):
    #try: 
    rx = re.compile('\W+')
    res = rx.sub(' ',string).strip()
    res = res.replace('\'','')
    return ' '.join([x.lemma_ for x in nlp(res) if not x.is_stop ])

    #except TypeError:
        #print('TypeError',string)

In [6]:
df['q1_clean'] = df['question1'].apply(lambda x: clean_data(x))
df['q2_clean'] = df['question2'].apply(lambda x: clean_data(x))

In [7]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_clean,q2_clean
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,step step guide invest share market india,step step guide invest share market
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,story kohinoor koh noor diamond,happen indian government steal kohinoor koh no...
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,increase speed internet connection vpn,internet speed increase hack dns
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,mentally lonely solve,find remainder math 23 24 math divide 24 23
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,dissolve water quikly sugar salt methane carbo...,fish survive salt water


In [8]:
print('q1_counts \n',df['q1_clean'].apply(lambda x: len(x.split())).describe())
print('q2_counts \n',df['q2_clean'].apply(lambda x: len(x.split())).describe())

q1_counts 
 count    404288.000000
mean          5.222468
std           2.913420
min           0.000000
25%           3.000000
50%           5.000000
75%           6.000000
max          68.000000
Name: q1_clean, dtype: float64
q2_counts 
 count    404288.000000
mean          5.278838
std           3.182402
min           0.000000
25%           3.000000
50%           4.000000
75%           6.000000
max          92.000000
Name: q2_clean, dtype: float64


In [ ]:
word_vecs =np.array([word.vector for word in doc for doc in nlp.pipe(df['question1'], n_threads=50)])

In [10]:
def build_vocab(string):
    for x in string.split():
        vocab.add(x)
        lexicon.append(x)
    return

In [11]:
vocab = set()
lexicon = []

In [12]:
_ = df['q1_clean'].apply(lambda x: build_vocab(x))
_ = df['q2_clean'].apply(lambda x: build_vocab(x))

In [13]:
vocab_size = len(vocab)

In [14]:
vocab_size

77092

In [15]:
def generate_labels(inputs, cnter,max_len,vocab_len):
    np_input = []
    
    for sentences in inputs:
        sen  = np.full(max_len,vocab_len)
        for idx,word in enumerate(sentences.split()):
            if(idx<max_len):
                sen[idx] = cnter.index(word)
        np_input.append(sen)
        
    return np.array(np_input).astype(np.int)
        
        

In [16]:
from collections import Counter
vocab_cnter = Counter(lexicon)

In [17]:
sorted_vocab = [x[0] for x in vocab_cnter.most_common()]

In [18]:
sorted_vocab[:10]

['good', 's', 'india', 'like', 'people', 'way', 't', 'quora', 'learn', 'life']

In [19]:
word2vec = []
for word in sorted_vocab:
    word2vec.append(nlp(word).vector)

In [20]:
len(word2vec)

77092

In [21]:
word2vec = np.array(word2vec)
word2vec.shape

(77092, 300)

In [22]:
q1_inputs = generate_labels(df['q1_clean'],sorted_vocab,30,len(vocab))
print('q1 processed')
q2_inputs = generate_labels(df['q2_clean'],sorted_vocab,30,len(vocab))

q1 processed


In [ ]:
from keras.models import Model
from keras.layers.core import Dropout, Dense, Flatten, Activation
from keras.layers.embeddings import Embedding
from keras.layers.wrappers import Bidirectional
from keras.layers.recurrent import GRU,LSTM
from keras.layers.pooling import MaxPooling2D
from keras.layers import Concatenate,Input, concatenate
from keras import backend as K


def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

# Q1
input_1 = Input(shape=(30,))
embedd = Embedding(len(sorted_vocab),300,
                        weights =[word2vec],
                        name="Embedding_1")
embedding_1 = (embedd)(input_1)
bi_gru_1 = Bidirectional(LSTM(200,name='LSTM_1',dropout=.8,recurrent_dropout=.8))(embedding_1)

#Q2 
input_2 = Input(shape=(30,))
embedding_2 = (embedd)(input_2)
bi_gru_2 = Bidirectional(LSTM(200,name="LSTM_2",dropout=.8,recurrent_dropout=.8))(embedding_2)
#Merge
#concat = Concatenate(axis=1)
merged = concatenate([bi_gru_1,bi_gru_2])
#merged = (Flatten())(merged)
#Dense
dense = Dense(4098,activation='relu',name='dense')(merged)
drop_1 = Dropout(.5,name='drop_1')(dense)
dense_2 = Dense(1000,activation='relu',name='dense_2')(drop_1)
drop_2 = Dropout(.5,name='drop_2')(dense_2)
pred = Dense(2,activation="softmax",name='pred')(drop_2)
model = Model(inputs=[input_1,input_2],outputs=pred)
# model = Sequential()
# model.add(Embedding(len(vocab),64,input_length=sentence_len,name='Embedding_Input'))
# model.add(Bidirectional(GRU(100,dropout=.5,name='GRU')))
# model.add(Dense(1000,activation='relu',))

In [24]:
q1_inputs.shape

(404288, 30)

In [25]:
q2_inputs.shape

(404288, 30)

In [27]:

def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))


In [ ]:
from keras import optimizers
adam = optimizers.Adam(lr=.001)
model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])
model.fit(x=[q1_inputs,q2_inputs],y=pd.get_dummies(df['is_duplicate']).values,batch_size=128,epochs=30,validation_split=.1)

Train on 363859 samples, validate on 40429 samples
Epoch 1/30
363859/363859 [==============================] - 351s - loss: 0.5601 - acc: 0.7176 - val_loss: 0.4949 - val_acc: 0.7626
Epoch 2/30
363859/363859 [==============================] - 346s - loss: 0.5006 - acc: 0.7579 - val_loss: 0.4702 - val_acc: 0.7696
Epoch 3/30
363859/363859 [==============================] - 346s - loss: 0.4721 - acc: 0.7731 - val_loss: 0.4653 - val_acc: 0.7716
Epoch 4/30
363859/363859 [==============================] - 345s - loss: 0.4495 - acc: 0.7841 - val_loss: 0.4817 - val_acc: 0.7735
Epoch 5/30
363859/363859 [==============================] - 347s - loss: 0.4330 - acc: 0.7920 - val_loss: 0.4603 - val_acc: 0.7745
Epoch 6/30
363859/363859 [==============================] - 346s - loss: 0.4185 - acc: 0.7993 - val_loss: 0.4660 - val_acc: 0.7765
Epoch 7/30
363859/363859 [==============================] - 348s - loss: 0.4079 - acc: 0.8054 - val_loss: 0.4664 - val_acc: 0.7773
Epoch 8/30
363859/363859 [======